In [35]:
print("OK")

OK


In [36]:
%pwd

'c:\\Users\\Dell\\Desktop\\Projects\\MedBot\\MedBot\\reserach'

In [10]:
import os 
os.chdir("../")
%pwd


'c:\\Users\\Dell\\Desktop\\Projects\\MedBot\\MedBot'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter




In [6]:
def load_pdf_file(data):
    loader=DirectoryLoader(data,glob="*.pdf",loader_cls=PyPDFLoader)
    documents=loader.load()
    return documents
        

In [11]:
data_dir = os.path.join(os.path.dirname(__file__), "..", "Data")
extracted_data=load_pdf_file(data_dir)

NameError: name '__file__' is not defined

In [41]:
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks


In [ ]:
print(extracted_data)  
text_chunks=text_split(extracted_data)
print(len(text_chunks))

In [43]:
from langchain.embeddings import HuggingFaceEmbeddings

In [44]:
def downlaod_hugging_face_embeddgings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [ ]:
embeddings=downlaod_hugging_face_embeddgings()
query_result=(embeddings.embed_query("Hello world!"));
print("length",len(query_result))

In [ ]:
query_result

In [ ]:
from pinecone import Pinecone, ServerlessSpec
from dotenv import load_dotenv
import os

load_dotenv()    

PINE_CONE_API_KEY = os.environ.get('PINE_CONE_API_KEY')
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')




In [ ]:
pc = Pinecone(api_key=PINE_CONE_API_KEY)
index_name = "quickstart"

pc.create_index(
    name="medbotindex",
    dimension=384, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [ ]:
import os 
os.environ["PINECONE_API_KEY"] = PINE_CONE_API_KEY;
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY;

In [54]:
from langchain_pinecone import PineconeVectorStore

docsearch=PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name="medbotindex",
    embedding=embeddings
)


In [55]:
from langchain_pinecone import PineconeVectorStore
docsearch=PineconeVectorStore.from_existing_index(
    index_name="medbotindex",
    embedding=embeddings
)

In [ ]:
docsearch

In [ ]:
retriver=docsearch.as_retriever(search_type="similarity",search_kwards={"k":3})
retriver_docs=retriver.invoke("Jeffery Archer")
retriver_docs

In [ ]:
from langchain_openai import OpenAI
llm=OpenAI(temperature=0.4,max_tokens=500)



In [64]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate



In [ ]:
system_prompt = (
    "You are a helpful assistant that helps the user to find the answer to their question. "
    "You are given a context and you have to answer the question based on the context. "
    "Use the following pieces of context to answer the question at the end. "
    "If you don't know the answer, just say that you don't know. Don't try to make up an answer. "
    "{context}"
)

prompt=ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])

question_answer_chain=create_stuff_documents_chain(
    llm=llm,
    prompt=prompt
)

rag_chain=create_retrieval_chain(
    retriever=retriver,
    combine_docs_chain=question_answer_chain
)




In [ ]:

response=rag_chain.invoke({"input":"Can you create a character sketch for George?"})
print(response["answer"])

In [ ]:
from src.helper import load_pdf_file, text_split, downlaod_hugging_face_embeddgings
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
from dotenv import load_dotenv
import os




load_dotenv()    
PINE_CONE_API_KEY = os.environ.get('PINE_CONE_API_KEY')
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')


extracted_data=load_pdf_file(data="Data/")
text_chunks=text_split(extracted_data)
embeddings=downlaod_hugging_face_embeddgings()

pc = Pinecone(api_key=PINE_CONE_API_KEY)
index_name = "medbotindex"

pc.create_index(
    name=index_name,
    dimension=384, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

docsearch=PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings
)